# 📊 Комплексный сбор данных с Binance

Этот notebook собирает **максимально полную историю всех доступных данных** с Binance и объединяет их в один DataFrame.

## 🎯 Что собирается:

1. **OHLCV данные** (Open, High, Low, Close, Volume)
2. **Рыночные покупки и продажи** (Aggregated Trades)
3. **Лимитные заявки (Order Book)** на глубинах 3%, 8%, 15%, 60%
4. **Ликвидации** (для фьючерсов)
5. **Открытый интерес** (Open Interest)
6. **Вычисляемые метрики** (buy/sell ratio, liquidation ratio)

---

## ⚠️ Важно:

- Сбор полной истории (с 2017 года) может занять несколько часов
- Для тестирования рекомендуется начать с небольшого периода (например, последний месяц)
- Order Book данные собираются периодическими снимками (Binance не предоставляет историю)


## 📦 Установка зависимостей

Если вы запускаете notebook впервые, выполните эту ячейку для установки необходимых библиотек.


In [ ]:
# Раскомментируйте следующую строку если нужно установить зависимости
!pip install python-binance pandas numpy requests tqdm


## 🔧 Импорт библиотек и настройка


In [2]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Добавляем путь к проекту
sys.path.append(str(Path().absolute().parent))

# Импорт из проекта
from binance_data_collector.scripts.collect_comprehensive_data import ComprehensiveBinanceCollector
from binance_data_collector.config import config
from binance_data_collector.utils.file_handler import save_data

print("✅ Библиотеки загружены успешно!")
print(f"📁 Рабочая директория: {Path().absolute()}")
print(f"📊 Директория для данных: {config.DATA_DIR}")


✅ Библиотеки загружены успешно!
📁 Рабочая директория: c:\Users\XE\Desktop\CRYPTO_2025\binance_data_collector
📊 Директория для данных: c:\Users\XE\Desktop\CRYPTO_2025\binance_data_collector\data


## ⚙️ Настройка параметров сбора

Настройте параметры для сбора данных:


In [3]:
# ===== ПАРАМЕТРЫ СБОРА ДАННЫХ =====

# Торговая пара
SYMBOL = "BTCUSDT"  # Например: BTCUSDT, ETHUSDT, BNBUSDT

# Период сбора данных
START_DATE = datetime(2024, 1, 1)  # Начальная дата
END_DATE = datetime.now()          # Конечная дата (сегодня)

# Интервалы
INTERVAL = "15m"           # Интервал сбора OHLCV (1m, 5m, 15m, 1h, 1d и т.д.)
TARGET_INTERVAL = "1h"     # Целевой интервал агрегации (15m, 1h, 4h, 1d и т.д.)

# Опции
SAVE_TO_FILE = True        # Сохранять ли данные в файл

print("📊 Параметры сбора данных:")
print(f"   Символ: {SYMBOL}")
print(f"   Период: {START_DATE.date()} - {END_DATE.date()}")
print(f"   Интервал сбора: {INTERVAL}")
print(f"   Целевой интервал: {TARGET_INTERVAL}")
print(f"   Сохранение: {'Да' if SAVE_TO_FILE else 'Нет'}")


📊 Параметры сбора данных:
   Символ: BTCUSDT
   Период: 2024-01-01 - 2025-11-02
   Интервал сбора: 15m
   Целевой интервал: 1h
   Сохранение: Да


## 🚀 Инициализация коллектора

Создаем экземпляр коллектора для сбора данных.


In [4]:
# Создаем коллектор
collector = ComprehensiveBinanceCollector(symbol=SYMBOL)

print(f"✅ Коллектор инициализирован для {SYMBOL}")
print(f"📊 Глубины order book: {collector.orderbook_depths}")


✅ Коллектор инициализирован для BTCUSDT
📊 Глубины order book: [0.03, 0.08, 0.15, 0.6]


## 📥 Сбор данных

**Внимание**: Это может занять значительное время в зависимости от выбранного периода.

Собираются:
1. OHLCV данные (основа)
2. Агрегированные сделки (рыночные объемы)
3. Ликвидации
4. Открытый интерес
5. Order Book (периодические снимки)


In [ ]:
# Запускаем сбор данных
print("🚀 Начинаем сбор данных...")
print("=" * 70)

df = collector.collect_comprehensive_data(
    start_date=START_DATE,
    end_date=END_DATE,
    interval=INTERVAL,
    target_interval=TARGET_INTERVAL
)

print("\n" + "=" * 70)
if not df.empty:
    print("✅ Сбор данных завершен успешно!")
else:
    print("❌ Не удалось собрать данные!")


2025-11-02 15:19:32 - comprehensive_collector - INFO - ======================================================================
2025-11-02 15:19:32 - comprehensive_collector - INFO - 🚀 КОМПЛЕКСНЫЙ СБОР ДАННЫХ С BINANCE
2025-11-02 15:19:32 - comprehensive_collector - INFO - ======================================================================
2025-11-02 15:19:32 - comprehensive_collector - INFO - 📊 Символ: BTCUSDT
2025-11-02 15:19:32 - comprehensive_collector - INFO - 📅 Период: 2024-01-01 - 2025-11-02
2025-11-02 15:19:32 - comprehensive_collector - INFO - ⏱️  Интервал сбора: 15m
2025-11-02 15:19:32 - comprehensive_collector - INFO - ⏱️  Целевой интервал: 1h
2025-11-02 15:19:32 - comprehensive_collector - INFO - ⏰ Это может занять значительное время...
2025-11-02 15:19:32 - comprehensive_collector - INFO - ======================================================================
2025-11-02 15:19:32 - comprehensive_collector - INFO - 
📊 Шаг 1/5: Сбор OHLCV данных...
2025-11-02 15:19:32 - comp

🚀 Начинаем сбор данных...


OHLCV (15m): 65batch [00:43,  1.50batch/s]                     
2025-11-02 15:20:16 - comprehensive_collector - INFO - ✅ Получено 64478 свечей OHLCV
2025-11-02 15:20:16 - comprehensive_collector - INFO - 
📊 Шаг 2/5: Сбор агрегированных сделок (рыночные объемы)...
2025-11-02 15:20:16 - comprehensive_collector - INFO - 📊 Сбор агрегированных сделок с 2024-01-01 по 2025-11-02
Aggregated Trades:   8%|▊         | 39447/483579 [5:11:25<57:30:19,  2.15batch/s] 

## 📊 Просмотр результатов

Проверяем собранные данные:


In [ ]:
if not df.empty:
    print(f"📊 Размер данных: {len(df)} строк × {len(df.columns)} колонок")
    print(f"📅 Период данных: {df.index[0]} - {df.index[-1]}")
    print(f"\n📋 Список колонок ({len(df.columns)}):")
    for i, col in enumerate(df.columns, 1):
        print(f"   {i:2d}. {col}")
    
    print("\n" + "=" * 70)
    print("📈 Первые 10 строк:")
    print("=" * 70)
    display(df.head(10))
    
    print("\n" + "=" * 70)
    print("📊 Основная статистика:")
    print("=" * 70)
    display(df.describe())
else:
    print("❌ Нет данных для отображения")


## 💾 Сохранение данных

Сохраняем данные в CSV файл:


In [ ]:
if not df.empty and SAVE_TO_FILE:
    # Генерируем имя файла
    start_str = df.index[0].strftime('%Y%m%d')
    end_str = df.index[-1].strftime('%Y%m%d')
    filename = f"{SYMBOL}_comprehensive_{TARGET_INTERVAL}_{start_str}_{end_str}.csv"
    filepath = config.DATA_DIR / "historical" / filename
    
    # Сохраняем
    print(f"💾 Сохранение данных в {filepath}...")
    save_data(df, filepath, format="csv")
    
    print(f"\n✅ Данные успешно сохранены!")
    print(f"📁 Файл: {filepath}")
    print(f"📊 Размер: {len(df)} строк × {len(df.columns)} колонок")
else:
    if df.empty:
        print("⚠️ Нет данных для сохранения")
    else:
        print("⚠️ Сохранение отключено (SAVE_TO_FILE = False)")


## 📈 Анализ данных (примеры)

Несколько примеров анализа собранных данных:


In [ ]:
if not df.empty:
    # Визуализация (требует matplotlib)
    try:
        import matplotlib.pyplot as plt
        import matplotlib.dates as mdates
        
        fig, axes = plt.subplots(3, 1, figsize=(14, 10))
        
        # График 1: Цена
        axes[0].plot(df.index, df['close'], label='Close Price', linewidth=1)
        axes[0].set_title(f'{SYMBOL} - Цена закрытия', fontsize=12, fontweight='bold')
        axes[0].set_ylabel('Цена (USDT)')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        # График 2: Объемы
        if 'volume' in df.columns:
            axes[1].bar(df.index, df['volume'], alpha=0.6, label='Volume')
            axes[1].set_title('Объемы торгов', fontsize=12, fontweight='bold')
            axes[1].set_ylabel('Объем (BTC)')
            axes[1].legend()
            axes[1].grid(True, alpha=0.3)
        
        # График 3: Рыночные покупки/продажи
        if 'market_buy_volume' in df.columns and 'market_sell_volume' in df.columns:
            axes[2].plot(df.index, df['market_buy_volume'], label='Market Buy Volume', alpha=0.7)
            axes[2].plot(df.index, df['market_sell_volume'], label='Market Sell Volume', alpha=0.7)
            axes[2].set_title('Рыночные объемы покупок и продаж', fontsize=12, fontweight='bold')
            axes[2].set_ylabel('Объем')
            axes[2].set_xlabel('Дата')
            axes[2].legend()
            axes[2].grid(True, alpha=0.3)
        
        # Форматирование дат на оси X
        for ax in axes:
            ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
            ax.xaxis.set_major_locator(mdates.AutoDateLocator())
            plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)
        
        plt.tight_layout()
        plt.show()
        
    except ImportError:
        print("⚠️ matplotlib не установлен. Для визуализации выполните: pip install matplotlib")
    except Exception as e:
        print(f"⚠️ Ошибка при построении графиков: {e}")


## 🔍 Дополнительный анализ

Дополнительные метрики и анализ:


In [ ]:
if not df.empty:
    print("=" * 70)
    print("📊 ДОПОЛНИТЕЛЬНЫЕ МЕТРИКИ")
    print("=" * 70)
    
    # Изменение цены
    if 'close' in df.columns:
        df['price_change_pct'] = df['close'].pct_change() * 100
        print(f"\n💰 Изменение цены:")
        print(f"   Среднее изменение: {df['price_change_pct'].mean():.4f}%")
        print(f"   Максимальное изменение: {df['price_change_pct'].max():.4f}%")
        print(f"   Минимальное изменение: {df['price_change_pct'].min():.4f}%")
    
    # Рыночные объемы
    if 'market_buy_volume' in df.columns and 'market_sell_volume' in df.columns:
        total_buy = df['market_buy_volume'].sum()
        total_sell = df['market_sell_volume'].sum()
        print(f"\n📈 Рыночные объемы:")
        print(f"   Всего покупок: {total_buy:,.2f} BTC")
        print(f"   Всего продаж: {total_sell:,.2f} BTC")
        if total_sell > 0:
            print(f"   Соотношение покупок/продаж: {total_buy/total_sell:.4f}")
    
    # Order Book дисбаланс (если есть данные)
    ob_columns = [col for col in df.columns if 'imbalance' in col]
    if ob_columns:
        print(f"\n📊 Order Book дисбаланс:")
        for col in ob_columns:
            avg_imbalance = df[col].mean()
            print(f"   {col}: {avg_imbalance:.4f} (среднее)")
    
    # Ликвидации
    if 'total_liquidations' in df.columns:
        total_liq = df['total_liquidations'].sum()
        if total_liq > 0:
            print(f"\n💥 Ликвидации:")
            print(f"   Всего ликвидаций: {total_liq:,.2f} BTC")
            if 'long_liquidations' in df.columns:
                long_liq = df['long_liquidations'].sum()
                short_liq = df['short_liquidations'].sum()
                print(f"   Long: {long_liq:,.2f} BTC ({long_liq/total_liq*100:.1f}%)")
                print(f"   Short: {short_liq:,.2f} BTC ({short_liq/total_liq*100:.1f}%)")
    
    print("\n" + "=" * 70)


## 💡 Полезные советы

### Для быстрого тестирования:
- Используйте небольшой период (например, последний месяц)
- Установите большие интервалы (`1h` или `1d`)

### Для полного анализа:
- Собирайте данные с максимальной историей (с 2017 года)
- Используйте меньшие интервалы (`15m`, `1h`) для детального анализа

### Работа с большими объемами данных:
- Собирайте данные по частям (например, по годам)
- Сохраняйте промежуточные результаты
- Используйте `parquet` формат для экономии места

### Загрузка сохраненных данных:
```python
from binance_data_collector.utils.file_handler import load_data
df = load_data('data/historical/your_file.csv')
```
